<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [ ]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 1362 (delta 96), reused 82 (delta 42), pack-reused 1212 (from 2)
Receiving objects: 100% (1362/1362), 13.64 MiB | 25.54 MiB/s, done.
Resolving deltas: 100% (884/884), done.


In [ ]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_lits
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import  trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [ ]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 4) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.2:6881] v1 unspecified system error "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.2:6881] v1 unspecified system error "" (1)
7.10% complete (down: 109959.1 kB/s up: 3136.7 kB/s peers: 10) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://academictorrents.com/announce.php)[1

## Model

In [ ]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT_liver.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<ipython-input-7-a1e217a1c2e2>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))


<All keys matched successfully>

In [ ]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model.load_state_dict({
    k: v for k, v in model_stage_1.state_dict().items() if "out" not in k
}, strict=False)

_IncompatibleKeys(missing_keys=['out.conv.weight', 'out.conv.bias'], unexpected_keys=[])

## Load data

In [ ]:
full_train_dataset, val_dataset = get_datasets_lits(source_folder=config["source_folder_lits"], seed=123, fold_number=2, mode="tumor",model_stage_1=model_stage_1)
print(len(full_train_dataset), len(val_dataset))

105 26


In [ ]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=0)

## Training

In [ ]:
criterion = TverskyLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [ ]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = config["max_epochs"],
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=True,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/5 1/105 loss: 0.7221 time 7.35s
Epoch 1/5 2/105 loss: 0.7390 time 8.10s
Epoch 1/5 3/105 loss: 0.7549 time 7.24s
Epoch 1/5 4/105 loss: 0.7629 time 6.77s
Epoch 1/5 5/105 loss: 0.7374 time 4.07s
Epoch 1/5 6/105 loss: 0.7368 time 6.02s
Epoch 1/5 7/105 loss: 0.7396 time 3.70s
Epoch 1/5 8/105 loss: 0.7381 time 6.79s
Epoch 1/5 9/105 loss: 0.7425 time 5.49s
Epoch 1/5 10/105 loss: 0.7470 time 3.44s
Epoch 1/5 11/105 loss: 0.7496 time 1.98s
Epoch 1/5 12/105 loss: 0.7509 time 8.15s
Epoch 1/5 13/105 loss: 0.7501 time 5.96s
Epoch 1/5 14/105 loss: 0.7520 time 1.59s
Epoch 1/5 15/105 loss: 0.7534 time 1.98s
Epoch 1/5 16/105 loss: 0.7545 time 6.70s
Epoch 1/5 17/105 loss: 0.7538 time 2.51s
Epoch 1/5 18/105 loss: 0.7548 time 2.02s
Epoch 1/5 19/105 loss: 0.7554 time 6.50s
Epoch 1/5 20/105 loss: 0.7485 time 3.00s
Epoch 1/5 21/105 loss: 0.7494 time 1.56s
Epoch 1/5 22/105 loss: 0.7463 time 2.67s
Epoch 1/5 23/105 loss: 0.7475 time 6.22s
Epoch 1/5 24/105 loss: 0.7486 time 4.44s
Epoch 1/5 25/105 loss: 0.

Final training epochs: 1/5 ---[loss: 0.7339] ---[time 457.26s]
INFO:kltn.utils.logger:Final training epochs: 1/5 ---[loss: 0.7339] ---[time 457.26s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/5 105/105 loss: 0.7339 time 5.08s


Val 1/5 1/26, Dice_Tumor: 0.000000, time 2.37s
INFO:kltn.utils.logger:Val 1/5 1/26, Dice_Tumor: 0.000000, time 2.37s
Val 1/5 2/26, Dice_Tumor: 0.000000, time 1.61s
INFO:kltn.utils.logger:Val 1/5 2/26, Dice_Tumor: 0.000000, time 1.61s
Val 1/5 3/26, Dice_Tumor: 0.000000, time 6.73s
INFO:kltn.utils.logger:Val 1/5 3/26, Dice_Tumor: 0.000000, time 6.73s
Val 1/5 4/26, Dice_Tumor: 0.000000, time 8.25s
INFO:kltn.utils.logger:Val 1/5 4/26, Dice_Tumor: 0.000000, time 8.25s
Val 1/5 5/26, Dice_Tumor: 0.000000, time 1.24s
INFO:kltn.utils.logger:Val 1/5 5/26, Dice_Tumor: 0.000000, time 1.24s
Val 1/5 6/26, Dice_Tumor: 0.000000, time 5.40s
INFO:kltn.utils.logger:Val 1/5 6/26, Dice_Tumor: 0.000000, time 5.40s
Val 1/5 7/26, Dice_Tumor: 0.000000, time 6.39s
INFO:kltn.utils.logger:Val 1/5 7/26, Dice_Tumor: 0.000000, time 6.39s
Val 1/5 8/26, Dice_Tumor: 0.000000, time 4.61s
INFO:kltn.utils.logger:Val 1/5 8/26, Dice_Tumor: 0.000000, time 4.61s
Val 1/5 9/26, Dice_Tumor: 0.000000, time 1.18s
INFO:kltn.utils.l

...

New best (0.016707 --> 0.038250). At epoch 4
Epoch 5/5 1/105 loss: 0.7481 time 1.49s
Epoch 5/5 2/105 loss: 0.6415 time 6.32s
Epoch 5/5 3/105 loss: 0.6687 time 3.08s
Epoch 5/5 4/105 loss: 0.6867 time 7.12s
Epoch 5/5 5/105 loss: 0.6808 time 5.94s
Epoch 5/5 6/105 loss: 0.6882 time 1.74s
Epoch 5/5 7/105 loss: 0.6803 time 3.73s
Epoch 5/5 8/105 loss: 0.6759 time 1.84s
Epoch 5/5 9/105 loss: 0.6633 time 8.19s
Epoch 5/5 10/105 loss: 0.6676 time 1.14s
Epoch 5/5 11/105 loss: 0.6734 time 2.98s
Epoch 5/5 12/105 loss: 0.6760 time 5.69s
Epoch 5/5 13/105 loss: 0.6730 time 1.61s
Epoch 5/5 14/105 loss: 0.6670 time 5.74s
Epoch 5/5 15/105 loss: 0.6481 time 5.71s
Epoch 5/5 16/105 loss: 0.6382 time 10.42s
Epoch 5/5 17/105 loss: 0.6437 time 5.27s
Epoch 5/5 18/105 loss: 0.6274 time 3.92s
Epoch 5/5 19/105 loss: 0.6293 time 7.55s
Epoch 5/5 20/105 loss: 0.6158 time 2.56s
Epoch 5/5 21/105 loss: 0.6027 time 2.11s
Epoch 5/5 22/105 loss: 0.6081 time 6.54s
Epoch 5/5 23/105 loss: 0.6125 time 6.17s
Epoch 5/5 24/105 los

Final training epochs: 5/5 ---[loss: 0.6058] ---[time 474.76s]
INFO:kltn.utils.logger:Final training epochs: 5/5 ---[loss: 0.6058] ---[time 474.76s]

********************Epoch 5 Validation********************
INFO:kltn.utils.logger:
********************Epoch 5 Validation********************


Epoch 5/5 105/105 loss: 0.6058 time 6.87s


Val 5/5 1/26, Dice_Tumor: 0.000000, time 1.49s
INFO:kltn.utils.logger:Val 5/5 1/26, Dice_Tumor: 0.000000, time 1.49s
Val 5/5 2/26, Dice_Tumor: 0.317383, time 1.35s
INFO:kltn.utils.logger:Val 5/5 2/26, Dice_Tumor: 0.317383, time 1.35s
Val 5/5 3/26, Dice_Tumor: 0.000000, time 7.48s
INFO:kltn.utils.logger:Val 5/5 3/26, Dice_Tumor: 0.000000, time 7.48s
Val 5/5 4/26, Dice_Tumor: 0.000000, time 7.22s
INFO:kltn.utils.logger:Val 5/5 4/26, Dice_Tumor: 0.000000, time 7.22s
Val 5/5 5/26, Dice_Tumor: 0.201575, time 1.30s
INFO:kltn.utils.logger:Val 5/5 5/26, Dice_Tumor: 0.201575, time 1.30s
Val 5/5 6/26, Dice_Tumor: 0.374379, time 6.38s
INFO:kltn.utils.logger:Val 5/5 6/26, Dice_Tumor: 0.374379, time 6.38s
Val 5/5 7/26, Dice_Tumor: 0.000000, time 6.16s
INFO:kltn.utils.logger:Val 5/5 7/26, Dice_Tumor: 0.000000, time 6.16s
Val 5/5 8/26, Dice_Tumor: 0.000000, time 5.56s
INFO:kltn.utils.logger:Val 5/5 8/26, Dice_Tumor: 0.000000, time 5.56s
Val 5/5 9/26, Dice_Tumor: 0.543407, time 1.22s
INFO:kltn.utils.l

New best (0.038250 --> 0.055505). At epoch 5


In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)